In [182]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from skimage import io
import matplotlib.pyplot as plt
import prep_ecg
from torchvision.transforms import ToTensor
import utils
import time

In [147]:
# data=m4p.loadmat('DREAMER.mat')['DREAMER']['Data']
# baseline=baseline_ds(data)
# stimuli=stimuli_ds(data)
# score=score_dict(data)
# for i in range(0,18):
#     select_n_slice(0,i,baseline,10,'cwt',True,score)

In [148]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [172]:
ecg_image=torchvision.datasets.ImageFolder(root='images/',transform=ToTensor())
train=torch.utils.data.DataLoader(ecg_image,batch_size=2,shuffle=True,num_workers=0)
(ecg_image.class_to_idx)

{'Happiness': 0, 'Neutral': 1, 'Peacefulness': 2, 'Sad': 3, 'Scary': 4}

In [180]:
# %%time
# a=0
# for i,(b,c) in enumerate(train):
#     print(i)
#     print('---')
#     print(b[0].size())
#     print('---')
#     print(c)

In [173]:
class ecgnet_one(nn.Module):
    def __init__(self):
        
        super(ecgnet_one, self).__init__()
        
        self.conv1a=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, padding=1)
        self.conv1b=nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, padding=1)
        self.pool1=nn.MaxPool2d(2,2)
        
        self.conv2a=nn.Conv2d(in_channels=24,out_channels=96,kernel_size=3,padding=1)
        self.conv2b=nn.Conv2d(in_channels=96,out_channels=96,kernel_size=3,padding=1)
        self.pool2=nn.MaxPool2d(2,2)
        
        self.conv3a=nn.Conv2d(in_channels=96,out_channels=256,kernel_size=3,padding=1)
        self.conv3b=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1)
        self.pool3=nn.MaxPool2d(2,2)
        
        self.conv4=nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1)
        self.pool4=nn.MaxPool2d(2,2)
        
        self.linear1=nn.Linear(20480,20480)
        self.linear2=nn.Linear(20480,2048)
        self.linear3=nn.Linear(2048,5)
        
    def forward(self, x):

        x = self.conv1a(x)
        x = F.relu(x)
        x = self.conv1b(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2a(x)
        x = F.relu(x)
        x = self.conv2b(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv3a(x)
        x = F.relu(x)
        x = self.conv3b(x)
        x = F.relu(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool4(x)

        x = x.view(-1, 20480)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x) 
        
        return x

In [174]:
%%time
net=ecgnet_one()
utils.display_num_param(net)
net=net.to(device)

There are 463505293 (463.51 million) parameters in this neural network
CPU times: user 2.67 s, sys: 561 ms, total: 3.23 s
Wall time: 3.23 s


In [175]:
criterion=nn.CrossEntropyLoss()
my_lr=0.25

In [183]:
start=time.time()
for epoch in range(1,20):
    if epoch ==10 or epoch==14 or epoch==18:
        my_lr=my_lr/2
    optimizer=torch.optim.SGD(net.parameters(),lr=my_lr)
    running_loss=0
    running_error=0
    for num,(image,label) in enumerate(train):
        batch_image=image.to(device)
        batch_label=label.to(device)
        inputs=batch_image
        inputs.requires_grad_()
        scores=net(inputs)
        loss=criterion(scores,batch_label)
        loss.backward()
        optimizer.step()
        running_loss+=loss.detach().item()
        error=utils.get_error(scores.detach(),batch_label)
        running_error+=error.item()
    total_loss = running_loss/num
    total_error = running_error/num
    elapsed = (time.time()-start)/60
    
    print('epoch=',epoch, '\t time=', elapsed,'min','\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    print('')

epoch= 1 	 time= 2.0104305466016132 min 	 lr= 0.25 	 loss= nan 	 error= 84.90566037735849 percent

epoch= 2 	 time= 3.901526180903117 min 	 lr= 0.25 	 loss= nan 	 error= 84.90566037735849 percent

epoch= 3 	 time= 5.817395933469137 min 	 lr= 0.25 	 loss= nan 	 error= 84.90566037735849 percent



KeyboardInterrupt: 